In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model2 = models.Sequential([
    layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    layers.Dense(84, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

test_loss, test_acc = model2.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


170498071/170498071 [==============================] - 11s 0us/step
Epoch 1/5
625/625 [==============================] - 30s 47ms/step - loss: 1.6990 - accuracy: 0.3742 - val_loss: 1.4777 - val_accuracy: 0.4637
Epoch 2/5
625/625 [==============================] - 27s 44ms/step - loss: 1.4024 - accuracy: 0.4947 - val_loss: 1.3804 - val_accuracy: 0.5083
Epoch 3/5
625/625 [==============================] - 29s 47ms/step - loss: 1.2928 - accuracy: 0.5363 - val_loss: 1.3491 - val_accuracy: 0.5227
Epoch 4/5
625/625 [==============================] - 27s 44ms/step - loss: 1.2115 - accuracy: 0.5705 - val_loss: 1.2449 - val_accuracy: 0.5605
Epoch 5/5
313/313 [==============================] - 2s 8ms/step - loss: 1.1762 - accuracy: 0.5860
Test accuracy: 0.5860000252723694


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def modified_resnet_block(x, filters, kernel_size=3, stride=1, activation='relu'):
    y = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation(activation)(y)
    y = layers.Conv2D(filters, kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)
    if stride > 1:
        x = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
    out = layers.add([x, y])
    out = layers.Activation(activation)(out)
    return out

input_tensor = tf.keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(input_tensor)
x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
x = layers.Conv2D(128, 7, strides=2, padding='same', activation='relu')(x)
x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
x = modified_resnet_block(x, 128)
x = modified_resnet_block(x, 128)
x = layers.GlobalAveragePooling2D()(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_tensor, outputs=output_tensor)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


Epoch 1/5
625/625 [==============================] - 137s 215ms/step - loss: 1.4775 - accuracy: 0.4578 - val_loss: 1.7299 - val_accuracy: 0.4294
Epoch 2/5
625/625 [==============================] - 134s 215ms/step - loss: 1.1520 - accuracy: 0.5880 - val_loss: 1.1110 - val_accuracy: 0.6076
Epoch 3/5
625/625 [==============================] - 139s 223ms/step - loss: 1.0087 - accuracy: 0.6418 - val_loss: 1.2526 - val_accuracy: 0.5668
Epoch 4/5
625/625 [==============================] - 136s 218ms/step - loss: 0.9123 - accuracy: 0.6761 - val_loss: 0.9862 - val_accuracy: 0.6555
Epoch 5/5
313/313 [==============================] - 7s 22ms/step - loss: 1.0068 - accuracy: 0.6471
Test accuracy: 0.6470999717712402
